### Imports

In [1]:
import numpy as np
import pandas as pd 
from collections import Counter
import os
import glob
import copy

### Opening the CSV files 

In [2]:
dataframes = [pd.read_csv(file, sep=',') for file in sorted(glob.glob('../feature_tables' + "/*."+'csv'))]
modalities = [file.split(".")[0] for file in sorted(os.listdir('../feature_tables'))]

In [3]:
# make a dictionary that contains all modalities as a dataframe
all_features = dict()

for modal, df in zip(modalities, dataframes):
    table = modal.split(" - ")[1]
    all_features[table] = df
    
# drop irrelevant columns    
for moda in all_features:
    all_features[moda].drop(columns=['CURIE', 'Definition', 'Synonyms'], inplace=True)

### Read all the files for every cohort

In [4]:
datasets = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../cohort_studies_full_data/' + "/*."+'csv'))]
cohorts = [file.split(".")[0] for file in sorted(os.listdir('../cohort_studies_full_data/'))]

In [5]:
# make a dictionary that contains all modalities as a dataframe
cohort_studies = dict()
all_cohort_names = list() # save  a list of cohort names

for cohort, dataset in zip(cohorts, datasets):
    cohort_n = cohort.split("_MERGE")[0]
    all_cohort_names.append(cohort_n)
    cohort_studies[cohort_n] = dataset

In [6]:
# make the index column consistent among the cohort dataframes
for cohort in cohort_studies:
    if cohort!='JADNI':
        cohort_studies[cohort]['ID'] = cohort_studies[cohort].index
        cohort_studies[cohort] = cohort_studies[cohort].reset_index().set_index('ID')
        cohort_studies[cohort].dropna(axis=1, how='all', inplace=True) # drop columns with all NAN entries 

## Functions

### Check the total number of patients for each mapped feature

In [7]:
def patients_per_feature(feature_df, cohort_df, result_df):
    """read every cohort study dataframe from a dictionary of dataframes and check the number of participants
    that have measurements for every mapped feature. Return a dictionary of dataframe with modalities under 
    which we mapped the features. Note: the number of participants are normalized by the totol number 
    of participants at the baseline visit (i.e. Number of participant|number of participant/total).
    
    Note: while looking for a certain feature in a dataset there is a possibility that we would find multiple
    columns. We used substring search as the mapping features are the original name of the columns in the dataset
    but while merging the tables some features got a suffix. For example: if we search for AGE in ADNI.columns, 
    we would find  AGE, IMAGEUID_x, MOTHAGE, MOTHSXAGE, FATHAGE, FATHSXAGE, SIBAGE, SIBSXAGE, IMAGEUID_y. 
    Thus, we first look for AGE and if it could not be found then look for substring matches."""

    for modality in result_df.keys():
        result = result_df[modality]
        
        # check the intersecton to select the dataframes that exist in both dictionaries (all dataframe and mapping dictionaries)
        for cohort in feature_df[modality].columns.intersection(cohort_df.keys()):
            total = len(cohort_df[cohort].loc[cohort_df[cohort]['Months']==0].index.unique()) # total number of participants at baseline visit
            
            # select the features that were mapped for each cohort
            for feature in feature_df[modality][cohort].loc[feature_df[modality][cohort].notna()]:

                # if there is a comma it means there are two features mapped to one feature
                # if there is not a comma then the features are mepped one to one
                if "," not in feature:
                    # we have multiple targets for certain features in the columns, we look for 100% match first
                    flag=False 

                    # check the columns of each cohort study
                    for col in cohort_df[cohort].columns:

                        # if we cannot find the column name, there is chance the column names have a suffix
                        if feature==col:
                            flag=True # if we found the 100% match
                            visits = list(Counter(cohort_df[cohort]['Months'].loc[cohort_df[cohort]['Months'].notna()])) # make a list of all visits for each cohort
                    
                            # for every visit calculate the number of participant for every feature and normilize it 
                            for vis in visits:
                                col_new = feature_df[modality].loc[feature_df[modality][cohort]==feature, 'Feature']
                                patient = len(cohort_df[cohort].loc[cohort_df[cohort]['Months']==vis][col].dropna().index.unique())
                                result.loc[(result.index==cohort) & (result.Months==vis), col_new] = str(patient) + "|" + str(round(patient/total, 3))
                                
                        # when the feature was not found, search the columns using the substring matching
                        elif (feature in col) & (flag==False):
                            visits = list(Counter(cohort_df[cohort]['Months'].loc[cohort_df[cohort]['Months'].notna()])) # make a list of all visits for each cohort

                            # for every visit calculate the number of participant for every feature and normilize it 
                            for vis in visits:
                                col_new = feature_df[modality].loc[feature_df[modality][cohort]==feature, 'Feature']
                                patient = len(cohort_df[cohort].loc[cohort_df[cohort]['Months']==vis][col].dropna().index.unique())
                                result.loc[(result.index==cohort) & (result.Months==vis), col_new] = str(patient) + "|" + str(round(patient/total, 3))

                # if there is a comma it means there are two features mapped to one feature
                elif "," in feature:
                    # we have multiple targets for certain features in the columns, we look for 100% match first
                    flag=False

                    # check the columns of each cohort study
                    for col in cohort_df[cohort].columns:

                        # if we cannot find the column name, there is chance the column names have a suffix
                        if feature.split(", ")[0]==col:
                            flag=True # if we found the 100% match
                            visits = list(Counter(cohort_df[cohort]['Months'].loc[cohort_df[cohort]['Months'].notna()])) # make a list of all visits for each cohort

                            # for every visit calculate the number of participant for every feature and normilize it 
                            for vis in visits:
                                col_new = feature_df[modality].loc[feature_df[modality][cohort]==feature, 'Feature']
                                patient = len(cohort_df[cohort].loc[cohort_df[cohort]['Months']==vis][col].dropna().index.unique())
                                result.loc[(result.index==cohort) & (result.Months==vis), col_new] = str(patient) + "|" + str(round(patient/total, 3))
                                
                        # when the feature was not found, search the columns using the substring matching
                        elif (feature.split(", ")[0] in col) & (flag==False):
                            visits = list(Counter(cohort_df[cohort]['Months'].loc[cohort_df[cohort]['Months'].notna()])) # make a list of all visits for each cohort

                            # for every visit calculate the number of participant for every feature and normilize it 
                            for vis in visits:
                                col_new = feature_df[modality].loc[feature_df[modality][cohort]==feature, 'Feature']
                                patient = len(cohort_df[cohort].loc[cohort_df[cohort]['Months']==vis][col].dropna().index.unique())
                                result.loc[(result.index==cohort) & (result.Months==vis), col_new] = str(patient) + "|" + str(round(patient/total, 3))
                                
    # drop the rows that contain nan every column except the visit number(i.e. Months) column                        
    for moda in result_df:
        result_df[moda].replace({"0|0.0": np.nan}, inplace=True) # replace the entries that are zero with NAN, the feature exist but not for that visit
        subset_cols = list() # make a list of all columns but the visit as it is never NAN

        for i in result_df[moda].columns:

            if i!='Months':
                subset_cols.append(i)

        # drop rows with NAN entries for every column
        result_df[moda].dropna(how='all', subset=subset_cols, inplace=True)

In [8]:
def feature_exist(feature_df, cohort_df, result_df):
    """ check whether evey mapped feature exist in the respective cohort and rank it with "1" and if it does not exist 
    then rank that cell with "0". Return two dataframes one showing only the features that did not exist in the dataframe and 
    a complete one with both ranks.
    
    Note: while looking for a certain feature in a dataset there is a possibility that we would find multiple
    columns. We used substring search as the mapping features are the original name of the columns in the dataset
    but while merging the tables some features got a suffix. For example: if we search for AGE in ADNI.columns, 
    we would find  AGE, IMAGEUID_x, MOTHAGE, MOTHSXAGE, FATHAGE, FATHSXAGE, SIBAGE, SIBSXAGE, IMAGEUID_y. 
    Thus, we first look for AGE and if it could not be found then look for substring matches."""
    
    # make an empty dataframe for storing the features that do not exist in the actual dataset
    nonexistence = pd.DataFrame(columns=['cohort', 'feature'])

    for modality in result_df.keys():
        result = result_df[modality]

        # check the intersecton to select the dataframes that exist in both dictionaries (all dataframe and mapping dictionaries)
        for cohort in feature_df[modality].columns.intersection(cohort_df.keys()):

            # select the features that were mapped for each cohort
            for feature in feature_df[modality][cohort].loc[feature_df[modality][cohort].notna()]:
                
                # if there is a comma it means there are two features mapped to one feature
                # if there is not a comma then the features are mepped one to one
                if "," not in feature:
                    # we have multiple targets for certain features in the columns, we look for 100% match first
                    flag=False
                    ind = feature_df[modality].loc[feature_df[modality][cohort]==feature, 'Feature']
                    result.loc[ind, cohort] = 0 # fist write zero for every feature that we mapped in result dataframes

                    # check the columns of each cohort study
                    for col in cohort_df[cohort].columns:

                        if feature==col:
                            flag=True # if we found the 100% match
                            ind = feature_df[modality].loc[feature_df[modality][cohort]==feature, 'Feature']
                            result.loc[ind, cohort] = 1 # store rank one where the feature exist in the cohort dataset
                            
                        elif (feature in col) & (flag==False):
                            ind = feature_df[modality].loc[feature_df[modality][cohort]==feature, 'Feature']
                            result.loc[ind, cohort] = 1 # store rank one where the feature exist in the cohort dataset
                
                # if there is a comma it means there are two features mapped to one feature
                else:
                    # we have multiple targets for certain features in the columns, we look for 100% match first
                    flag=False
                    ind = feature_df[modality].loc[feature_df[modality][cohort]==feature, 'Feature']
                    result.loc[ind, cohort] = 0 # fist write zero for every feature that we mapped in result dataframes
                    feat1 = feature.split(", ")[0] # split the mapped features
                    feat2 = feature.split(", ")[1]

                    # check the columns of each cohort study
                    for col in cohort_df[cohort].columns:

                        # if one feature exist so does the other as these features are related
                        if (feat1==col) or (feat2==col):
                            flag=True # if we found the 100% match
                            ind = feature_df[modality].loc[feature_df[modality][cohort]==feature, 'Feature']
                            result.loc[ind, cohort] = 1 # store rank one where the feature exist in the cohort dataset
                        
                        elif (feat1 in col) or (feat2 in col) & (flag==False):
                            ind = feature_df[modality].loc[feature_df[modality][cohort]==feature, 'Feature']
                            result.loc[ind, cohort] = 1 # store rank one where the feature exist in the cohort dataset
                            
                            
    # from the result dataframe select the features that are rank zero and write it into another dataframe
    for moda in result_df:
        
        for col in result_df[moda].columns:
            bad = result_df[moda].loc[result_df[moda][col]==0].index.to_list() # find the nonexistence features
            
            # if there is at least one entry with the rank zero for each dataframe
            if len(bad)!=0:
                
                for i in bad:
                    bad_feat = feature_df[moda].loc[feature_df[moda]['Feature']==i, col].item()
                    nonexistence = nonexistence.append({'cohort': col, 'feature': bad_feat}, ignore_index=True)          
    
                    
    return nonexistence

In [9]:
def patient_measurment_availability(features, dataset, result_df):
    """ write a dictionary of cohort studies where each key represents the respective cohort and value of that key
    is a dataframe with patients as rows and all mapped features as columns. check for every patient whether the measurments
    were collected at any vist of the study. store rank '1' were the measurement exist for that participant.
    
    Note: while looking for a certain feature in a dataset there is a possibility that we would find multiple
    columns. We used substring search as the mapping features are the original name of the columns in the dataset
    but while merging the tables some features got a suffix. For example: if we search for AGE in ADNI.columns, 
    we would find  AGE, IMAGEUID_x, MOTHAGE, MOTHSXAGE, FATHAGE, FATHSXAGE, SIBAGE, SIBSXAGE, IMAGEUID_y. 
    Thus, we first look for AGE and if it could not be found then look for substring matches."""
        
    for cohort in dataset:
        df = dataset[cohort]
        
        # take the patient ids from the cohort as index column for the results dataframe
        result_df[cohort]['ID'] = df.index.unique() 
        result_df[cohort].set_index('ID', inplace=True)
        
        # for every feature find the mapped feature for the respective dataframe
        for i in result_df[cohort].columns:
            the_col = features.loc[features['Feature']==i, cohort].values[0]
            
            # if there is a comma it means there are two features mapped to one feature
            # if there is not a comma then the features are mepped one to one
            if ", " not in the_col:
                # we have multiple targets for certain features in the columns, we look for 100% match first
                flag=False

                for colu in df.columns:
                    
                    # if we cannot find the column name, there is chance the column names have a suffix
                    if the_col==colu:
                        flag=True # if we found the 100% match
                        sub_df = df.loc[df[colu].notna(), colu]

                        for patient in sub_df.index:
                            result_df[cohort].loc[patient, i] = 1 # rank one for the patients that have measurements for the feature
                    
                    # when the feature was not found, search the columns using the substring matching
                    elif (the_col in colu) & (flag==False):
                        sub_df = df.loc[df[colu].notna(), colu]

                        for patient in sub_df.index:
                            result_df[cohort].loc[patient, i] = 1 # rank one for the patients that have measurements for the feature
                            
            # if there is a comma it means there are two features mapped to one feature
            elif ", " in the_col:
                # we have multiple targets for certain features in the columns, we look for 100% match first
                flag=False
                
                for colu in df.columns:
                    
                    # if we cannot find the column name, there is chance the column names have a suffix
                    if the_col.split(", ")[0]==colu:
                        flag=True # if we found the 100% match
                        sub_df = df.loc[df[colu].notna(), colu]

                        for patient in sub_df.index:
                            result_df[cohort].loc[patient, i] = 1 # rank one for the patients that have measurements for the feature
                    
                    # when the feature was not found, search the columns using the substring matching
                    elif (the_col.split(", ")[0] in colu) & (flag==False):
                        sub_df = df.loc[df[colu].notna(), colu]

                        for patient in sub_df.index:
                            result_df[cohort].loc[patient, i] = 1 # rank one for the patients that have measurements for the feature        
                

        # replace the patient ids with 0 to n for data protection 
        result_df[cohort].set_index(np.arange(0, len(result_df[cohort])), inplace=True)
        result_df[cohort].index.name = 'ID'

### Results

### Create empty dictionaries of dataframes to store the result

In [10]:
# dictionary of dataframes (one dataframe per modality)
feature_availability = {name: pd.DataFrame(index = all_features[name]['Feature'], columns = all_cohort_names) for name in all_features.keys()}

# combine all the feature tables (modalities) into one big dataframe  
combined_mapping = pd.concat(all_features, ignore_index=False)
# make a dictionary with cohorts as key and empty dataframe as values with all mapped features for columns of each cohort
patient_vs_feature = {name: pd.DataFrame(columns = combined_mapping.loc[combined_mapping[name].notna(), 'Feature']) for name in combined_mapping.columns[1:21]}


# make a dictionary of dataframes where modalities are the keys and dataframes are the value
num_patients_per_visit = {name: pd.DataFrame(columns = all_features[name]['Feature']) for name in all_features.keys()}

for moda in num_patients_per_visit:
    
    for cohort in cohort_studies:
        # make a list of visit timepoints for each cohort
        vis_num = list(Counter(cohort_studies[cohort]['Months'].loc[cohort_studies[cohort]['Months'].notna()]))
        
        # write the cohort name and visit month as index
        for vis in vis_num:
            num_patients_per_visit[moda].loc[cohort + '__' + str(vis)] = np.nan
            
            
# divide the name and timepoints and store it into separated columns
for modal in num_patients_per_visit:
    num_patients_per_visit[modal]['cohorts'] = num_patients_per_visit[modal].index
    num_patients_per_visit[modal][['Cohort', 'Months']] = num_patients_per_visit[modal]['cohorts'].str.split('__', 1, expand=True)
    num_patients_per_visit[modal].drop(columns=['cohorts'], inplace=True) # set the cohort names as index
    num_patients_per_visit[modal] = num_patients_per_visit[modal].set_index('Cohort')
    
# change the month column to numeric     
for moda in num_patients_per_visit:
    num_patients_per_visit[moda]['Months'] = num_patients_per_visit[moda]['Months'].astype(str).astype(float)

### Call the Functions

In [11]:
patients_per_feature(all_features, cohort_studies, num_patients_per_visit)

In [12]:
bad_features = feature_exist(all_features, cohort_studies, feature_availability)

In [13]:
patient_measurment_availability(combined_mapping, cohort_studies, patient_vs_feature)

# write to tsv

In [14]:
for modal in num_patients_per_visit:
    num_patients_per_visit[modal].to_csv("number_of_patient_per_visit/" + f"{modal}.tsv", sep='\t', index_label='Cohort')

In [15]:
for modal in feature_availability:
    feature_availability[modal].to_csv("feature_availability_in_cohorts/" + f"{modal}.tsv", sep='\t', index_label='Feature')

In [16]:
for cohort in patient_vs_feature:
    patient_vs_feature[cohort].to_csv("feature_vs_patient_per_cohort/" + f"{cohort}.tsv", sep='\t', index_label='ID')

In [17]:
bad_features.to_csv("feature_availability_in_cohorts/nonexistence_features.tsv", sep='\t', index_label=False)